<a href="https://colab.research.google.com/github/pavansai26/cuda-programming-for-multiplication/blob/master/Copy_of_CUDA_programming_for_multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

import os
os.chdir('/content/gdrive/My Drive')
!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive


In [0]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [0]:
! pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
# plug in gives interface

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-htcw_buk
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-htcw_buk
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=9887103a862e51a27b2d224b50942899ebed8933080ef51f6497d553c7f39aee
  Stored in directory: /tmp/pip-ephem-wheel-cache-2pfu2tzc/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [0]:
%%cuda --name helloword.cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>

#define LOG_INPUT if(0)
#define LOG_OUTPUT if(1)
#define LOG if(0)


__global__ void hadamard(float *A, float *B, float *C, int M, int N)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x; # this gives id through the thread
    int j = blockDim.y * blockIdx.y + threadIdx.y;
    int index = i*N+j; # it gives memory address of the data we need to take
    //printf("%d\n",index);
    if (index < M*N)
    {
        C[index] = A[index] * B[index];
    }
}

/**
 * Host main routine
 */
void print_matrix(float *A,int m,int n)
{
    for(int i =0;i<m;i++)
    {
        for(int j=0;j<n;j++)
            printf("%.2f ",A[i*n+j]);
        printf("\n");
    }

}
int main(void)
{
    // Error code to check return values for CUDA calls
    cudaError_t err = cudaSuccess;

    // Print the vector length to be used, and compute its size
    
    int t;
    scanf("%d",&t);
    while(t--)
    {
        int m,n;
        scanf("%d %d",&m,&n);
        size_t size = m*n * sizeof(float);
        LOG printf("[Hadamard product of two matrices ]\n");

        // Allocate the host input vector A
        float *h_A = (float *)malloc(size);
        // Allocate the host input vector B
        float *h_B = (float *)malloc(size);
        // Allocate the host output vector C
        float *h_C = (float *)malloc(size);

        // Verify that allocations succeeded
        if (h_A == NULL || h_B == NULL || h_C == NULL)
        {
            fprintf(stderr, "Failed to allocate host vectors!\n");
            exit(EXIT_FAILURE);
        }

        // Initialize the host input vectors
        
        for (int i = 0; i < n*m; ++i)
        {
            scanf("%f",&h_A[i]);
            scanf("%f",&h_B[i]);

        }
        
        
        // Allocate the device input vector A
        float *d_A = NULL;
        err = cudaMalloc((void **)&d_A, size);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to allocate device vector A (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Allocate the device input vector B
        float *d_B = NULL;
        err = cudaMalloc((void **)&d_B, size);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to allocate device vector B (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Allocate the device output vector C
        float *d_C = NULL;
        err = cudaMalloc((void **)&d_C, size);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to allocate device vector C (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Copy the host input vectors A and B in host memory to the device input vectors in
        // device memory
        LOG printf("Copy input data from the host memory to the CUDA device\n");
        err = cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to copy vector A from host to device (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        err = cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to copy vector B from host to device (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        
        dim3 threadsPerBlock(32,32,1);
        dim3 blocksPerGrid(m/32,n/32,1);
        
        LOG printf("Number of blocks %d %d\n",m/256,n/256);
        hadamard<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, m, n);
        err = cudaGetLastError();

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to launch vectorAdd kernel (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Copy the device result vector in device memory to the host result vector
        // in host memory.
        LOG printf("Copy output data from the CUDA device to the host memory\n");
        err = cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to copy vector C from device to host (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Verify that the result vector is correct
        for (int i = 0; i < n*m; ++i)
        {
            if (fabs(h_A[i] * h_B[i] - h_C[i]) > 1e-5)
            {
                fprintf(stderr, "Result verification failed at element %d!\n", i);
                exit(EXIT_FAILURE);
            }
        }

        LOG printf("Test PASSED\n");

        // Free device global memory
        err = cudaFree(d_A);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to free device vector A (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        err = cudaFree(d_B);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to free device vector B (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        err = cudaFree(d_C);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to free device vector C (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Free host memory
        free(h_A);
        free(h_B);
        free(h_C);

        // Reset the device and exit
        // cudaDeviceReset causes the driver to clean up all state. While
        // not mandatory in normal operation, it is good practice.  It is also
        // needed to ensure correct operation when the application is being
        // profiled. Calling cudaDeviceReset causes all profile data to be
        // flushed before the application exits
        err = cudaDeviceReset();

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to deinitialize the device! error=%s\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }
        print_matrix(h_C,m,n);
        LOG printf("Done\n");
    }
    return 0;
}



'File written in /content/gdrive/My Drive/src/helloword.cu'

In [0]:
#!nvcc /content/src/helloword.cu -o /content/src/helloword
!pwd
!nvcc "/content/gdrive/My Drive/src/helloword.cu" -o "/content/gdrive/My Drive/src/helloword.cu"

/content/gdrive/My Drive
/content/gdrive/My Drive/src/helloword.cu(12): error: "#" not expected here

/content/gdrive/My Drive/src/helloword.cu(12): error: expected an expression

/content/gdrive/My Drive/src/helloword.cu(14): error: identifier "j" is undefined

/content/gdrive/My Drive/src/helloword.cu(14): error: "#" not expected here

/content/gdrive/My Drive/src/helloword.cu(14): error: expected an expression

/content/gdrive/My Drive/src/helloword.cu(20): warning: parsing restarts here after previous syntax error

/content/gdrive/My Drive/src/helloword.cu(20): error: expected a ";"

6 errors detected in the compilation of "/tmp/tmpxft_00000687_00000000-8_helloword.cpp1.ii".


In [0]:
!ls

'10 SIGNAL AND SYSTEMS.PDF'
'11 MEASUREMENTS PART 1 (1).PDF'
'11 MEASUREMENTS PART 2.PDF'
'12  POWER ELECTRONICS PART 1.PDF'
'12 POWER ELECTRONICS PART 2.PDF'
'1.DIGITAL ELECTRONICS AND MICROPROCESSOR  WORK BOOK-1.PDF'
'1.DIGITAL ELECTRONICS THEORY WITH PRACTICE QUESTIONS.PDF'
'1_List Manual_for_6RA80.pdf'
'2.ANALOG  ELECTRONICS WORK BOOK-1.PDF'
'2.NETWORK  CIRCUITS THEORY WITH PRACTISE BOOK.PDF'
'3.MATHS APTITUDE AND ENGLISH-1.PDF'
'3. MICROPROCESSOR THEORY WITH PRACTICE QUESTIONS.PDF'
 4.EMF.PDF
'4.NETWORKS AND CONTROL SYSTEMS WORK BOOK-1.PDF'
 5_6154406436852990046.pdf
'5.CONTROL SYSTEMS MISSED PAGE 331.PDF'
'5.CONTROL SYSTEMS PART 1.PDF'
'5. CONTROL SYSTEMS PART 2.PDF'
'5. CONTROL SYSTEMS PART 3.PDF'
'5.CONTROL SYSTEMS PART 4.PDF'
'5.CONTROL SYSTEMS PART 5.PDF'
'5.ELECTRICAL MACHINES AND ELECTROMAGNETIC THEORY WORK BOOK-1.PDF'
'6.COMMUNICATIONS,MATERIALS,COMPUTER FUNDEMENTALS WORK BOOK-1.PDF'
'6. MATERIAL SCIENCES THEORY WITH PRACTISE.PDF'
'7.ANALOG ELECTRONICS THEORY WITH PRACTISE

In [0]:
!cd src

!pwd

/content/gdrive/My Drive


In [0]:
!"/content/gdrive/My Drive/sample_input.txt" < sample_input.txt

/bin/bash: sample_input.txt: No such file or directory


In [0]:
%%cuda --name matmul.cu
#include <algorithm>
#include <cassert>
#include <cstdlib>
#include <functional>
#include <iostream>
#include <vector>

using std::cout;
using std::generate;
using std::vector;

__global__ void matrixMul(const int *a, const int *b, int *c, int N) {
  // Compute each thread's global row and column index
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  // Iterate over row, and down column
  c[row * N + col] = 0;
  for (int k = 0; k < N; k++) {
    // Accumulate results for a single element
    c[row * N + col] += a[row * N + k] * b[k * N + col];
  }
}

// Check result on the CPU
void verify_result(vector<int> &a, vector<int> &b, vector<int> &c, int N) {
  // For every row...
  for (int i = 0; i < N; i++) {
    // For every column...
    for (int j = 0; j < N; j++) {
      // For every element in the row-column pair
      int tmp = 0;
      for (int k = 0; k < N; k++) {
        // Accumulate the partial results
        tmp += a[i * N + k] * b[k * N + j];
      }

      // Check against the CPU result
      assert(tmp == c[i * N + j]);
    }
  }
}

int main() {
  // Matrix size of 1024 x 1024;
  int N = 1 << 10;

  // Size (in bytes) of matrix
  size_t bytes = N * N * sizeof(int);

  // Host vectors
  vector<int> h_a(N * N);
  vector<int> h_b(N * N);
  vector<int> h_c(N * N);

  // Initialize matrices
  generate(h_a.begin(), h_a.end(), []() { return rand() % 100; });
  generate(h_b.begin(), h_b.end(), []() { return rand() % 100; });

  // Allocate device memory
  int *d_a, *d_b, *d_c;
  cudaMalloc(&d_a, bytes);
  cudaMalloc(&d_b, bytes);
  cudaMalloc(&d_c, bytes);

  // Copy data to the device
  cudaMemcpy(d_a, h_a.data(), bytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, h_b.data(), bytes, cudaMemcpyHostToDevice);

  // Threads per CTA dimension
  int THREADS = 32;

  // Blocks per grid dimension (assumes THREADS divides N evenly)
  int BLOCKS = N / THREADS;

  // Use dim3 structs for block  and grid dimensions
  dim3 threads(THREADS, THREADS);
  dim3 blocks(BLOCKS, BLOCKS);

  // Launch kernel
  matrixMul<<<blocks, threads>>>(d_a, d_b, d_c, N);

  // Copy back to the host
  cudaMemcpy(h_c.data(), d_c, bytes, cudaMemcpyDeviceToHost);

  // Check result
  verify_result(h_a, h_b, h_c, N);

  cout << "COMPLETED SUCCESSFULLY\n";

  // Free memory on device
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  return 0;
}



'File written in /content/gdrive/My Drive/src/matmul.cu'

In [0]:
!nvcc "/content/gdrive/My Drive/src/matmul.cu" -o "/content/gdrive/My Drive/src/matmul"

In [0]:
!"/content/gdrive/My Drive/Vivek-PC-2015/vsinghal/business/Disrupt4.0/AI Lab/demos/AI Hardware/Accelerated Linear Algebra/src/matmul" < sample_input.txt

COMPLETED SUCCESSFULLY


In [0]:
%%cuda --name cublas.cu
#include <cublas_v2.h>
#include <curand.h>
#include <cassert>
#include <cmath>
#include <ctime>
#include <iostream>
#include <vector>

// Verify our result on the CPU
// Indexing must account for the CUBLAS operating on column-major data
void verify_solution(float *a, float *b, float *c, int M, int N, int K) {
  // Tolerance for our result (floats are imperfect)
  float epsilon = 0.001f;

  // For every row...
  for (int row = 0; row < M; row++) {
    // For every column
    for (int col = 0; col < N; col++) {
      // For every element in the row-col pair...
      float temp = 0;
      for (int i = 0; i < K; i++) {
        temp += a[row + M * i] * b[col * K + i];
      }

      // Check to see if the difference falls within our tolerance
      assert(fabs(c[col * M + row] - temp) <= epsilon);
    }
  }
}

int main() {
  // Dimensions for our matrices
  // MxK * KxN = MxN
  const int M = 1 << 9;
  const int N = 1 << 8;
  const int K = 1 << 7;

  // Pre-calculate the size (in bytes) of our matrices
  const size_t bytes_a = M * K * sizeof(float);
  const size_t bytes_b = K * N * sizeof(float);
  const size_t bytes_c = M * N * sizeof(float);

  // Vectors for the host data
  std::vector<float> h_a(M * K);
  std::vector<float> h_b(K * N);
  std::vector<float> h_c(M * N);
  
  // Allocate device memory
  float *d_a, *d_b, *d_c;
  cudaMalloc(&d_a, bytes_a);
  cudaMalloc(&d_b, bytes_b);
  cudaMalloc(&d_c, bytes_c);

  // Pseudo random number generator
  curandGenerator_t prng;
  curandCreateGenerator(&prng, CURAND_RNG_PSEUDO_DEFAULT);

  // Set the seed
  curandSetPseudoRandomGeneratorSeed(prng, (unsigned long long)clock());

  // Fill the matrix with random numbers on the device
  curandGenerateUniform(prng, d_a, M * K);
  curandGenerateUniform(prng, d_b, K * M);

  // cuBLAS handle
  cublasHandle_t handle;
  cublasCreate(&handle);

  // Scalaing factors
  float alpha = 1.0f;
  float beta = 0.0f;

  // Calculate: c = (alpha*a) * b + (beta*c)
  // MxN = MxK * KxN
  // Signature: handle, operation, operation, M, N, K, alpha, A, lda, B, ldb,
  // beta, C, ldc
  cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, M, N, K, &alpha, d_a, M, d_b, K,
              &beta, d_c, M);

  // Copy back the three matrices
  cudaMemcpy(h_a.data(), d_a, bytes_a, cudaMemcpyDeviceToHost);
  cudaMemcpy(h_b.data(), d_b, bytes_b, cudaMemcpyDeviceToHost);
  cudaMemcpy(h_c.data(), d_c, bytes_c, cudaMemcpyDeviceToHost);

  // Verify solution
  verify_solution(h_a.data(), h_b.data(), h_c.data(), M, N, K);
  std::cout << "COMPLETED SUCCESSFULLY\n";

  // Free our memory
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  return 0;
}

'File written in /content/gdrive/My Drive/src/cublas.cu'

In [0]:
!nvcc "/content/gdrive/My Drive/src/cublas.cu" -o "/content/gdrive/My Drive/src/cublas"

/tmp/tmpxft_00000831_00000000-10_cublas.o: In function `main':
tmpxft_00000831_00000000-5_cublas.cudafe1.cpp:(.text+0x4cf): undefined reference to `curandCreateGenerator'
tmpxft_00000831_00000000-5_cublas.cudafe1.cpp:(.text+0x4e9): undefined reference to `curandSetPseudoRandomGeneratorSeed'
tmpxft_00000831_00000000-5_cublas.cudafe1.cpp:(.text+0x507): undefined reference to `curandGenerateUniform'
tmpxft_00000831_00000000-5_cublas.cudafe1.cpp:(.text+0x525): undefined reference to `curandGenerateUniform'
tmpxft_00000831_00000000-5_cublas.cudafe1.cpp:(.text+0x534): undefined reference to `cublasCreate_v2'
tmpxft_00000831_00000000-5_cublas.cudafe1.cpp:(.text+0x5b1): undefined reference to `cublasSgemm_v2'
collect2: error: ld returned 1 exit status
